In [16]:
import pandas as pd
from src.recsys_baseline import VSKNN_STAN
from src.recsys_rl import rl_recommender
from src.evaluation import evaluate_sessions, MRR, HitRate, StrategicHybrid
from src.utils import evaluation_results_to_csv

## load datasets

In [17]:
train = pd.read_csv('data/processed datasets/diginetica/interactions_train_tr.txt',
                    sep='\t')
display(train.head())
display(train.shape)

,Type,SessionId,ItemId,Time,Date,Datestamp,TimeO,ItemSupport
0,view,1,9654,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:01:15.848000+00:00,74
1,view,1,33043,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:02:53.912000+00:00,41
2,view,1,32118,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:04:03.569000+00:00,19
3,view,1,12352,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:05:29.870000+00:00,79
4,view,1,35077,1.462752e+09,2016-05-09,1.462752e+09,2016-05-09 00:06:30.072000+00:00,47


(102272, 8)

In [18]:
test = pd.read_csv('data/processed datasets/diginetica/interactions_test.txt',
                    sep='\t', nrows=200)
display(test.head())
display(test.shape)

,Type,SessionId,ItemId,Time,Date,Datestamp,TimeO,ItemSupport
0,view,289,125013,1.464221e+09,2016-05-26,1.464221e+09,2016-05-26 00:00:18.301000+00:00,7
1,view,289,64068,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:14:07.735000+00:00,6
2,view,289,133346,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:14:38.934000+00:00,6
3,view,289,198930,1.464222e+09,2016-05-26,1.464221e+09,2016-05-26 00:18:48.607000+00:00,7
4,view,302,36202,1.464221e+09,2016-05-26,1.464221e+09,2016-05-26 00:00:45.583000+00:00,23


(200, 8)

## Hybrid model  (reward: click=3, purchase=10)

### 5 interactions

In [19]:
model_1 = rl_recommender(
    n_history=5,
    file_path=r'trained agents\Diginetica\click=3, purchase=10\20240609_101255 - n_hist 5 state_size 5 action_size 568638.pth',
    state_size=5,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[5]) # after 5 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=3, purchase=10\20240609_101255 - n_hist 5 state_size 5 action_size 568638.pth
Pretrained model successfully loaded


In [20]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=5', reward_func='click=3, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@2:,0.018072
1,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@3:,0.020080
2,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@4:,0.021586
3,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@5:,0.023996
4,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@10:,0.029834
5,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@15:,0.031849
6,Diginetica,RL Agent n=5,"click=3, purchase=10",MRR@20:,0.032520
7,Diginetica,RL Agent n=5,"click=3, purchase=10",HitRate@1:,0.006024
8,Diginetica,RL Agent n=5,"click=3, purchase=10",HitRate@2:,0.030120
9,Diginetica,RL Agent n=5,"click=3, purchase=10",HitRate@3:,0.036145


### 10 interactions

In [21]:
model_1 = rl_recommender(
    n_history=10,
    file_path=r'trained agents\Diginetica\click=3, purchase=10\20240609_122056 - n_hist 10 state_size 10 action_size 568638.pth',
    state_size=10,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=3, purchase=10\20240609_122056 - n_hist 10 state_size 10 action_size 568638.pth
Pretrained model successfully loaded


In [22]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=10', reward_func='click=3, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@2:,0.009036
1,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@3:,0.011044
2,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@4:,0.011044
3,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@5:,0.011044
4,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@10:,0.011044
5,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@15:,0.011044
6,Diginetica,RL Agent n=10,"click=3, purchase=10",MRR@20:,0.011044
7,Diginetica,RL Agent n=10,"click=3, purchase=10",HitRate@1:,0.000000
8,Diginetica,RL Agent n=10,"click=3, purchase=10",HitRate@2:,0.018072
9,Diginetica,RL Agent n=10,"click=3, purchase=10",HitRate@3:,0.024096


### 15 interactions

In [23]:
model_1 = rl_recommender(
    n_history=15,
    file_path=r'trained agents\Diginetica\click=3, purchase=10\20240609_123637 - n_hist 15 state_size 15 action_size 568638.pth',
    state_size=15,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[15]) # after 15 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=3, purchase=10\20240609_123637 - n_hist 15 state_size 15 action_size 568638.pth
Pretrained model successfully loaded


In [24]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=15', reward_func='click=3, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@2:,0.0
1,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@3:,0.0
2,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@4:,0.0
3,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@5:,0.0
4,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@10:,0.0
5,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@15:,0.0
6,Diginetica,RL Agent n=15,"click=3, purchase=10",MRR@20:,0.0
7,Diginetica,RL Agent n=15,"click=3, purchase=10",HitRate@1:,0.0
8,Diginetica,RL Agent n=15,"click=3, purchase=10",HitRate@2:,0.0
9,Diginetica,RL Agent n=15,"click=3, purchase=10",HitRate@3:,0.0


### 100 interactions
In other words, the model almost only uses the first model to predict next interactions with items.

In [25]:
model_1 = rl_recommender(
    n_history=100,
    file_path=r'trained agents\Diginetica\click=3, purchase=10\20240609_134910 - n_hist 100 state_size 100 action_size 568638.pth',
    state_size=100,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[100]) # after 100 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=3, purchase=10\20240609_134910 - n_hist 100 state_size 100 action_size 568638.pth
Pretrained model successfully loaded


In [26]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=100', reward_func='click=3, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@2:,0.0
1,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@3:,0.0
2,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@4:,0.0
3,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@5:,0.0
4,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@10:,0.0
5,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@15:,0.0
6,Diginetica,RL Agent n=100,"click=3, purchase=10",MRR@20:,0.0
7,Diginetica,RL Agent n=100,"click=3, purchase=10",HitRate@1:,0.0
8,Diginetica,RL Agent n=100,"click=3, purchase=10",HitRate@2:,0.0
9,Diginetica,RL Agent n=100,"click=3, purchase=10",HitRate@3:,0.0


## Hybrid model (reward: click=0, purchase=10)

### 5 interactions

In [27]:
model_1 = rl_recommender(
    n_history=5,
    file_path=r'trained agents\Diginetica\click=0, purchase=10\20240627_190056 - n_hist 5 state_size 5 action_size 568638.pth',
    state_size=5,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[5]) # after 5 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=0, purchase=10\20240627_190056 - n_hist 5 state_size 5 action_size 568638.pth
Pretrained model successfully loaded


In [28]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=5', reward_func='click=0, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@2:,0.018072
1,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@3:,0.020080
2,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@4:,0.021586
3,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@5:,0.023996
4,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@10:,0.029834
5,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@15:,0.031849
6,Diginetica,RL Agent n=5,"click=0, purchase=10",MRR@20:,0.032520
7,Diginetica,RL Agent n=5,"click=0, purchase=10",HitRate@1:,0.006024
8,Diginetica,RL Agent n=5,"click=0, purchase=10",HitRate@2:,0.030120
9,Diginetica,RL Agent n=5,"click=0, purchase=10",HitRate@3:,0.036145


### 10 interactions

In [29]:
model_1 = rl_recommender(
    n_history=10,
    file_path=r'trained agents\Diginetica\click=0, purchase=10\20240627_190955 - n_hist 10 state_size 10 action_size 568638.pth',
    state_size=10,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=0, purchase=10\20240627_190955 - n_hist 10 state_size 10 action_size 568638.pth
Pretrained model successfully loaded


In [35]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=10', reward_func='click=0, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@2:,0.000000
1,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@3:,0.000000
2,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@4:,0.000000
3,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@5:,0.000000
4,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@10:,0.001004
5,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@15:,0.001004
6,Diginetica,RL Agent n=10,"click=0, purchase=10",MRR@20:,0.001004
7,Diginetica,RL Agent n=10,"click=0, purchase=10",HitRate@1:,0.000000
8,Diginetica,RL Agent n=10,"click=0, purchase=10",HitRate@2:,0.000000
9,Diginetica,RL Agent n=10,"click=0, purchase=10",HitRate@3:,0.000000


### 15 interactions

In [31]:
model_1 = rl_recommender(
    n_history=15,
    file_path=r'trained agents\Diginetica\click=0, purchase=10\20240627_191855 - n_hist 15 state_size 15 action_size 568638.pth',
    state_size=15,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[15]) # after 15 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=0, purchase=10\20240627_191855 - n_hist 15 state_size 15 action_size 568638.pth
Pretrained model successfully loaded


In [32]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=15', reward_func='click=0, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@2:,0.006024
1,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@3:,0.006024
2,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@4:,0.006024
3,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@5:,0.006024
4,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@10:,0.006024
5,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@15:,0.006024
6,Diginetica,RL Agent n=15,"click=0, purchase=10",MRR@20:,0.006024
7,Diginetica,RL Agent n=15,"click=0, purchase=10",HitRate@1:,0.006024
8,Diginetica,RL Agent n=15,"click=0, purchase=10",HitRate@2:,0.006024
9,Diginetica,RL Agent n=15,"click=0, purchase=10",HitRate@3:,0.006024


### 100 interactions
In other words, the model almost only uses the first model to predict next interactions with items.

In [33]:
model_1 = rl_recommender(
    n_history=100,
    file_path=r'trained agents\Diginetica\click=0, purchase=10\20240627_192759 - n_hist 100 state_size 100 action_size 568638.pth',
    state_size=100,
    action_size=568638,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=100,
                     sample_size=1000,
                     similarity='vec',
                    #    stan:  
                     lambda_spw=4.9,  
                     lambda_snh=80,  
                     lambda_inh=9.8,  
                    #    vsknn:  
                     lambda_ipw=4.9, 
                     lambda_idf=5
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[100]) # after 100 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents\Diginetica\click=0, purchase=10\20240627_192759 - n_hist 100 state_size 100 action_size 568638.pth
Pretrained model successfully loaded


In [34]:
metrics = [
    MRR(2),
    MRR(3),
    MRR(4),
    MRR(5),
    MRR(10),
    MRR(15),
    MRR(20),
    HitRate(1),
    HitRate(2),
    HitRate(3),
    HitRate(4),
    HitRate(5)]


output = evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)
evaluation_results_to_csv(output=output, dataset_name='Diginetica', algo_name='RL Agent n=100', reward_func='click=0, purchase=10')

START evaluation of  200  actions in  34  sessions
    eval process:  0  of  200  actions:  0.0  %
END evaluation


,Dataset,Algorithm,Reward Function,Metric,Value
0,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@2:,0.000000
1,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@3:,0.000000
2,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@4:,0.000000
3,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@5:,0.000000
4,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@10:,0.001004
5,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@15:,0.001004
6,Diginetica,RL Agent n=100,"click=0, purchase=10",MRR@20:,0.001004
7,Diginetica,RL Agent n=100,"click=0, purchase=10",HitRate@1:,0.000000
8,Diginetica,RL Agent n=100,"click=0, purchase=10",HitRate@2:,0.000000
9,Diginetica,RL Agent n=100,"click=0, purchase=10",HitRate@3:,0.000000
